# Результаты обучения

Ноутбук для заглушки, пока не будет добавлено логирование.

**Качество при Bicubic (точка отсчета)**:
* `PSNR=35.27 dB`
* `SSIM=0.8622`

# U-Net

## Датасет DeepRockSR-2D

### Baseline
#### Параметры архитектуры
* `base_channels=64`
* `depth=4`
* `bilinear=False`
* `norm=True`
* `dropout=0.0`

#### Гиперпараметры
* `lr=2e-4`
* `weight_decay=1e-4`
* `optimimizer=AdamW`
* `scheduler=None`
* `epochs=30` (ранняя останова на 23)
* `batch_size=8`
* `num_workers=8`
* `path_size=256`

Функция потерь: `nn.functional.l1_loss()`

Время обучения ~ 30 минут

Комплектующие:
* NVIDIA GeForce RTX 4080 SUPER
* AMD Ryzen 7700 8-Core


**Качество**:
* `PSNR=35.28`
* `SSIM=0.8954`

### test 1

**Гипотеза**: увеличение количества эпох с 30 до 100 даст прирост метрик

Измененные параметры: `epochs=100`, `batch_size=8`, `early_stop_patience=0`

Время обучения всего: `2:06:36`

Время обучения для одной эпохи: `0:01:15`

`python train.py --data_root DeepRockSR-2D --epochs_x2 100 --batch_size 8 --workers 8 --early_stop_patience 0`

**Качество**:
* `PSNR=36.47`
* `SSIM=0.8997`

### test 2

**Гипотеза**

Измененные параметры:
* `nn.functional.l1_loss()` заменен на комбинацию `nn.functional.l1_loss() + (1 - pytorch_msssim.ssim())`
* `epochs=100`, `batch_size=8`, `early_stop_patience=0`

Время обучения всего: `2:09:08`

Время обучения для одной эпохи: `0:01:17`

`python train.py --data_root DeepRockSR-2D --epochs_x2 100 --batch_size 8 --workers 8 --early_stop_patience 0`

**Качество**:
* `PSNR=36.48`
* `SSIM=0.9035`

### test 3

Измененные параметры:
* `nn.functional.l1_loss()` заменен на комбинацию `nn.functional.l1_loss() + (1 - pytorch_msssim.ssim())`
* `epochs=100`, `batch_size=8`, `early_stop_patience=0`
* Удален `BatchNorm2d` из `UNetConfig`: `norm=False`

Время обучения всего: ручная ранняя остановка, примерно `0:20:20`

Время обучения для одной эпохи: `0:01:00`

`python train.py --data_root DeepRockSR-2D --epochs_x2 100 --batch_size 8 --workers 8 --early_stop_patience 0`

**Качество**:
* `PSNR=29.66`
* `SSIM=0.8513`

Темные изображения стали "сеточными", светлые пиксели стали еще светлее.

Несмотря на ухудшение метрик, рассматривается возможность продолжить эксперименты без `BatchNorm2d`, но с другими отличиями.

### test 4

**Основная проблема от test 2**: модель получает мало контекста от краев изображений (эффект черных сеточек).

**Возможное решение**: добавить `padding=1`, `padding_mode='reflect'`.

Измененные параметры:
* `nn.functional.l1_loss()` заменен на комбинацию `nn.functional.l1_loss() + (1 - pytorch_msssim.ssim())`
* `epochs=50`, `batch_size=8`, `early_stop_patience=0`
* Добавлен `padding_mode='reflect'` в `ConvBNAct` в `unet2d.py`

Время обучения всего: ~ `1:00:00`

Время обучения для одной эпохи: `0:01:17`

**Качество**:
* `PSNR=35.82`
* `SSIM=0.8950`

### test 5

**Гипотеза**: BatchNorm вредит качеству.

**Возможное решение**: убрать BatchNorm на upsample-этапе.

Измененные параметры:
* `nn.functional.l1_loss()` заменен на комбинацию `nn.functional.l1_loss() + (1 - pytorch_msssim.ssim())`
* `epochs=100`, `batch_size=8`, `early_stop_patience=0`
* Добавлен `padding_mode='reflect'` в `ConvBNAct` в `unet2d.py`
* Убран BatchNorm на `upsample-этапе`
* Возвращен режим `bilinear=True` внутри `UNetConfig`

Время обучения всего: ~ `2:40:33`

Время обучения для одной эпохи: `0:01:36`

Изображения похожи структурно, но сильно светлее в темных пикселях. Исчез эффект сеточек (следствие `bilinear=True`). Из за этого метрики ниже среднего. 

**Качество**:
* `PSNR=21.29`
* `SSIM=0.7766`

### test 6

Измененные параметры:
* Все то же, что и в **test 5**, но убрана нормализация `Normalize()` в `build_pair_transform`
* В архитектуре добавлен residual head, что предположительно хорошо скажется на схождении
* Loss теперь `L1`
* Убрана операция `denorm01` для `pred` и `targ`
* `BatchNorm` возвращен в encoder и в decoder
* `SRLoss` адаптирован под эти изменения
* 5 эпох обучения.

Время обучения всего: ~ `00:08:35`

Время обучения для одной эпохи: `0:02:08`

**Качество**:
* `PSNR=37.33`
* `SSIM=0.9029`

### test 7

Измененные параметры:
* Все то же, что и в **test 6**, но `patch_size=None` (обучение на полных изображениях)

Время обучения всего: ~ `0:27:53`

Время обучения для одной эпохи: `0:06:04`

**Качество**:
* `PSNR=37.41`
* `SSIM=0.9033`

Незначительная прибавка к качеству за счет утроенного времени обучения.

### test 8

Измененные параметры:
* Все то же, что и в **test 7**, но loss теперь снова `nn.functional.l1_loss() + (1 - pytorch_msssim.ssim())`, где `w_l1=0.9`, `w_ssim=0.1`

Время обучения всего: ~ `0:27:44`

Время обучения для одной эпохи: `0:05:56`

**Качество**:
* `PSNR=37.41`
* `SSIM=0.9031`

Нет видимых изменений по сравнению с test 7.

### test 9

Измененные параметры:
* Все то же, что и в **test 8**, но добавлен и использован новый метод upsample: `PixelShuffle`
* Возвращен `patch_size=256`
* Возвращен `l1_loss`

Время обучения всего: ~ `0:08:39`

Время обучения для одной эпохи: `0:02:05`

**Качество**:
* `PSNR=37.33`
* `SSIM=0.9029`

Без видимых изменений.

### test 10

* Теперь разбиение на патчи работает корректно: все изображение делится на патчи и используется в обучении. Ранее лишь вырезался `RandomCrop()`, что убавляло доступную информацию.
* `epochs=5`, `batch_size=8`, `patch_size=128`
* `mse_loss`
* Убраны `BatchNorm`, `residual head`, все аугментации.

Время обучения всего: ~ `0:50:00`

Время обучения для одной эпохи: `0:10:00`

**Качество**:
* `PSNR=37.54`
* `SSIM=0.9044`

Несмотря на небольшое количество эпох и небольшое различие в метриках, изображение вышло более реалистичным. Далее будем повышать `epochs`, оптимизируем обучение.

### test 12

* `epochs=15`, `batch_size=32`, `patch_size=128`
* `mse_loss`
* `base_channels=64`

Время обучения всего: ~ `1:30:00`

Время обучения для одной эпохи: `0:6:30`

**Качество**:
* `PSNR=37.54`
* `SSIM=0.9044`

## X4 

### test 1

`python train.py --data_root C:/Users/Вячеслав/Documents/superresolution/DeepRockSR-2D --sign deeprock_x4 --epochs 50 --scheduler OneCycle --batch_size 90 --loss mse --patch_size 128 --lr 5e-4 --weight_decay 1e-6 --workers 8`

Время на эпоху: `00:02:15`

test PSNR: 30.78 dB | SSIM: 0.7033

train loss стабильно падает, но val loss стоит на месте

### test 2

`--data_root C:/Users/Вячеслав/Documents/superresolution/DeepRockSR-2D --sign deeprock_x4 --epochs 100 --scheduler Exponential --batch_size 50 --loss l1 --lr 1e-4 --weight_decay 1e-5 --workers 8 --base_channels 64 --do_flips True --do_blur False --precomputed_patches C:/Users/Вячеслав/Documents/superresolution/deeprock_x4_train_patches_ps128_u8.pt`

Время на эпоху: `0:05:57`

elapsed=`9:55:16`

Bicubic PSNR: 28.32 dB | SSIM: 0.6246

test PSNR: 29.03 dB | SSIM: 0.6583

## Датасет MRCCM2D

### test 1

Bicubic `PSNR: 21.43` dB | `SSIM: 0.8646`

* `epochs=100`, `batch_size=8`, `patch_size=256`
* Архитектура `UNet2D` идентична той, на которой обучалась DeepRock модель
* `l1_loss`
* `PixelShuffle`
* `BatchNorm` везде

Время обучения всего: `0:56:01`

Время обучения одной эпохи: `0:00:33`

**Качество:**
* `PSNR=38.35`
* `SSIM=0.9514`

Приходится искусственно делать сохраненные изображения ярче для наглядности.

### test 2

* `epochs=20`, `batch_size=8`, `patch_size=256`
* Архитектура `UNet2D` идентична той, на которой обучалась DeepRock модель
* `l1_loss`
* `PixelShuffle`
* Убран `BatchNorm`

Время обучения всего: `0:13:59`

Время обучения одной эпохи: `0:00:41`

**Качество:**
* `PSNR=37.55`
* `SSIM=0.9463`

Артефакты дублирования убраны ценой небольшого количества метрик, также яркость в некоторых местах выражается лучше.